# A modified Broyden-like quasi-Newton method for nonlinear equations.
## Presentado por:
- Jaime Alejandro Stack Sánchez
- Angel Antonio Méndez Hernández
## Resumen.

En este trabajo presentamos la implementación del método propuesto en el paper *A modified Broyden-like quasi-Newton method for nonlinear equations* propuesto por Weijun Zhou y Li Zhang y lo comparámos con el método propuesto por Li y Fukushima además de una versión simplificada de este mismo. Encontrámos mejoras significativas en funciones que suelen sufrir de desbordamiento.



## Marco teórico
La familia de métodos de Broyden para resolver ecuaciones no lineales tienen la estructura \begin{equation}
\tag{1}
x_{k+1} = x_k + d_k^B, \quad d_k^B = -B_k^{-1}F_k
\end{equation}
donde $B_k$ se actualiza dada la fórmula de Broyden \begin{equation}
\tag{2}
B_{k+1} = B_k+\theta_k\dfrac{(y_k-B_ks_k)s_k^t}{\Vert s_k\Vert^2}, \quad s_k=x_{k+1} -x_k,\quad y_k = F_{k+1} -F_k
\end{equation}
donde $\theta_k$ se elige tal que $|\theta_k-1|\leq \theta$ y $B_{k+1}$ es no singular para $\theta\in (0,1)$.

En la siguiente modificación del método se propone cambiar (1) por \begin{equation}
\tag{3}
x_{k+1} = x_k + d_k, \quad d_k = d_k^B + d_k^{MB}
\end{equation}
donde \begin{equation}
\tag{4}
d_k^B = -B_k^{-1}F_k,\quad d_k^{MB} = -B_k^{-1}F(z_k), \quad z_k = x_k + d_k
\end{equation} con las mismas actualizaciones para $B_{k}$ como en (2).

Además, se ultiliza una forma de búsqueda en línea libre de derivadas, propuesta igualmente por Li y Fukushima que aplicada al método de Broyden tiene convergencia global y superlineal.

## Pseudocódigo del método modificado.

![](https://drive.google.com/uc?id=1qkh5NkJB3RyUkDWH47REZG6SZSgJjG-x)
![](https://drive.google.com/uc?id=1UwryGP2VJxooIOxigHODvXgJZdnfXEKy)
![](https://drive.google.com/uc?id=1Q0LNN3Y-2wP_QE9eWxXVFQPK2snN5mey)


## Convergencia

![](https://drive.google.com/uc?id=1jUvM5nCj5GvzN2oB7AD8q5u8xG2YmbP8)
![](https://drive.google.com/uc?id=1ARCwIVPSsdmAB8uQB8L1AzsvTVmgJNXM)
![](https://drive.google.com/uc?id=1DSrMDvakP79jExSgMxwNxmXulLBNLZul)

Sobre la condición de independencia uniformemente lineal:

Para un $\eta >0$ y un conjunto de indices $\{k_i\}_{i=1}^n$ se tiene que$$\sigma_\min(Q_k) \geq \mu$$donde $Q_k = \left( \frac{s_{k_1}}{||s_{k_1}||}, \frac{s_{k_2}}{||s_{k_2}||}, ..., \frac{s_{k_n}}{||s_{k_n}||}\right)$.

## Código



In [ ]:
import numpy as np
from prettytable import PrettyTable

In [ ]:
def bt(x, d, fnc, fk, r, s2, e, maxIter = 1000):
  a = 1
  counter = 1
  while ((np.linalg.norm(fnc(x+a*d)) > fk - s2*np.dot(d,d)*(a**2.0) + e*fk) and (counter <= maxIter)):
    counter += 1
    a*=r
  return a

In [ ]:
# Método de Broyden "Normal"
def broyden(fnc, x0, B0, theta, tol, maxIter):
  xk = x0.copy()
  Bk = B0.copy()
  fk = fnc(xk)
  for k in range(maxIter):
    normfk = np.linalg.norm(fk)
    if normfk < tol:
      return xk, normfk, k+1, True
    # Calculamos dirección de movimiento
    if np.linalg.det(Bk) != 0:
      dk = np.linalg.solve(Bk, -fk)
    else:
      print("Terminamos por singularidad de matriz")
      return xk, normfk, k+1, True
    # Calculamos nuevo punto.
    xk1 = xk + dk
    # Actualizamos B
    fk1 = fnc(xk1)
    yk = fk1 - fk
    Bk = Bk + theta*np.outer(((yk - Bk@dk)/np.dot(dk, dk)),dk)
    # Actualizamos fk y xk
    xk = xk1
    fk = fk1
  return xk, np.linalg.norm(fnc(xk)), maxIter, False

In [ ]:
# Método de Broyden con tamaño de paso
# args = s1, s2, t, r, rho
def broyden_step(fnc, x0, B0, theta, tol, maxIter, *args):
  xk =  x0.copy()
  Bk = B0.copy()
  fk = fnc(xk)
  for k in range(maxIter):
    aux1 = np.linalg.norm(fk)
    if aux1 < tol:
      return xk, aux1, k+1, True
    # Calculamos dirección de movimiento y tamaño de paso
    if np.linalg.det(Bk) != 0:
      dk = np.linalg.solve(Bk, -fk)
    else:
      print("Matriz singular")
      return xk, aux1, k+1, True
    zk =  xk + dk
    fzk = fnc(zk)
    if np.linalg.norm(fzk) <= args[4]*aux1 - args[1]*np.dot(dk, dk):
      ak = 1
    else:
      e = 1.0/(k+1)**2
      ak = bt(xk, dk, fnc, aux1, args[3], args[1], e)
    # Obtenemos el nuevo xk
    xk1 = xk + ak*dk
    fk1 = fnc(xk1)
    yk = fk1 - fk
    sk = xk1 - xk
    Bk = Bk + theta*np.outer(((yk - Bk@sk)/np.dot(sk, sk)),sk)
    # Actualizamos fk y xk
    xk = xk1
    fk = fk1
  return xk, np.linalg.norm(fnc(xk)), maxIter, False

In [ ]:
# Método de Broyden Modificado
# args = s1, s2, t, r, rho
def broyden_modified(fnc, x0, B0, theta, tol, maxIter, *args):
  xk =  x0.copy()
  Bk = B0.copy()
  fk = fnc(xk)
  counter = 0
  for k in range(maxIter):
    aux1 = np.linalg.norm(fk)
    if aux1 < tol:
      return xk, aux1, k+1, True, counter
    # Calculamos dirección de movimiento y tamaño de paso
    if np.linalg.det(Bk) != 0:
      dk1 = np.linalg.solve(Bk, -fk)
    else:
      print("Matriz singular")
      return xk, aux1, k+1, True, counter
    zk =  xk + dk1
    fzk = fnc(zk)
    # Paso 3
    if np.linalg.norm(fzk) > args[2]*aux1:
      dk = dk1
      aux2 = (np.linalg.norm(fnc(xk+dk)) <= args[4]*aux1 - args[0]*np.dot(dk,dk))
      if aux2:
        ak = 1
      else:
        e = 1.0/(k+1)**2
        ak = bt(xk, dk, fnc, aux1, args[3], args[1], e)
    else:
      if np.linalg.det(Bk) != 0:
        dk2 = np.linalg.solve(Bk, -fzk)
      else:
        print("Matriz singular")
        return xk, aux1, k+1, True, counter
      aux = (np.linalg.norm(fnc(xk + dk1 + dk2)) <= args[4]*aux1 - args[0]*np.dot(dk1+dk2, dk1+dk2))
      if aux:
        dk = dk1+ dk2
        counter += 1
        ak = 1
      else:
        dk = dk1
        aux2 = (np.linalg.norm(fnc(xk+dk)) <= args[4]*aux1 - args[0]*np.dot(dk,dk))
        if aux2:
          ak = 1
        else:
          e = 1.0/(k+1)**2
          ak = bt(xk, dk, fnc, aux1, args[3], args[1], e)
    # Obtenemos el nuevo xk
    xk1 = xk + ak*dk
    fk1 = fnc(xk1)
    yk = fk1 - fk
    sk = xk1 - xk
    Bk = Bk + theta*np.outer(((yk - Bk@sk)/np.dot(sk, sk)),sk)
    # Actualizamos fk y xk
    xk = xk1
    fk = fk1
  return xk, np.linalg.norm(fnc(xk)), maxIter, False, counter

In [ ]:
def F1(x):
  f1 = 10*(x[1]-x[0]**2)
  f2 = 1-x[0]
  return np.array([f1, f2])

def JF1(x):
  m1 = -20*x[0]
  return np.array([[m1, 10.0], [-1, 0]])

def F2(x):
  f1 = x[0] + 10*x[1]
  f2 = np.sqrt(5)*(x[2]-x[3])
  f3= (x[1]-2*x[2])**2
  f4 = np.sqrt(10)*(x[0]-x[3])**2
  return np.array([f1,f2,f3,f4])

def F3(x):
  n = len(x)
  c = 1
  f = x.copy()
  for i in range(1, n+1):
    suma = 0
    mui = (i-0.5)/n
    for j in range(1, n+1):
      suma += mui*x[j-1]/(mui + (j-0.5)/n)
    f[i-1] -= 1/(1 - c*suma/(2*n))
  return f

def F4(x):
  n = len(x)
  M = 2*np.identity(n)
  for i in range(1, n):
    M[i-1][i] = M[i][i-1] = -1
  sin = (np.sin(x) - 1)/(n+1)**2
  return M@x + sin

def F5(x):
  n = len(x)
  f = np.zeros(n)
  f[0] = 2*x[0] + np.sin(x[0]) -1
  f[-1] = 2*x[-1] + np.sin(x[-1]) -1
  for i in range(1, n-1):
    f[i] = -2*x[i-1] + 2*x[i] +np.sin(x[i]) -1
  return f

def F6(x):
  n = len(x)
  f = np.zeros(n)
  f[0] = np.exp(x[0]) - 1
  for i in range(2, n+1):
    f[i-1] = i*(np.exp(x[i-1]) + x[i-2] -1)/10
  return f

def F7(x):
  n = len(x)
  f = np.zeros(n)
  f[0] = x[0]*(x[0]**2 + x[1]**2) - 1
  f[-1] = x[-1]*(x[-2]**2 + x[-1]**2)
  for i in range(1, n-1):
    f[i] = x[i]*(x[i-1]**2 +2*(x[i]**2) +x[i+1]**2)-1
  return f

In [ ]:
def print_results(beta, tol, maxIter, Bs, B_names, fnc, fnc_name, x0):
  table = PrettyTable()
  table.title = "Resultados del método de Broyden con {}".format(fnc_name)
  table.field_names = ["beta", "B_0", "M1 - Iter", "M1 - ||fk||", "M2 - Iter", "M2 - ||fk||", "M3 - Iter", "M3 - ||fk||"]
  for i, B in enumerate(Bs):
    for b in beta:
      x1 = (b*x0).copy()
      x2 = (b*x0).copy()
      B1 = B.copy()
      B2 = B.copy()
      sol1 = broyden(fnc, x1, B1, 1, tol, maxIter)
      sol2 = broyden_step(fnc, x2, B2, 1, tol, maxIter, 0.1, 0.1, 1, 0.1, np.sqrt(0.9))
      sol3 = broyden_modified(fnc, x2, B2, 1, tol, maxIter, 0.1, 0.1, 1, 0.1, np.sqrt(0.9))
      table.add_row([b, B_names[i], sol1[2], sol1[1], sol2[2], sol2[1], sol3[2], sol3[1]])
    table.add_row([" ", " ", " ", " ", " ", " ", " ", " "])
  print(table)

In [ ]:
def print_results1(beta, tol, maxIter, fnc, fnc_name, ns, opc = 1):
  table = PrettyTable()
  table.title = "Resultados del método de Broyden con {}".format(fnc_name)
  table.field_names = ["beta", "B_0", "n", "M1 - Iter", "M1 - ||fk||", "M2 - Iter", "M2 - ||fk||", "M3 - Iter", "M3 - ||fk||", "Nd"]
  for b in beta:
    for n in ns:
      B = np.identity(n)
      if opc == 1:
        x0 = np.array([1/i for i in range(1, n+1)])
      else:
        x0 = np.ones(n)
      x1 = (b*x0).copy()
      x2 = (b*x0).copy()
      B1 = B.copy()
      B2 = B.copy()
      sol1 = broyden(fnc, x1, B1, 1, tol, maxIter)
      sol2 = broyden_step(fnc, x2, B2, 1, tol, maxIter, 0.1, 0.1, 1, 0.1, np.sqrt(0.9))
      sol3 = broyden_modified(fnc, x2, B2, 1, tol, maxIter, 0.1, 0.1, 1, 0.1, np.sqrt(0.9))
      table.add_row([b, "I", n, sol1[2], sol1[1], sol2[2], sol2[1], sol3[2], sol3[1], sol3[4]])
    table.add_row([" ", " ", " "," "," ", " ", " ", " ", " ", " "])
  print(table)

#**Pruebas**
Para estas primeras pruebas, consideramos lo siguiente:

*   $Jx^*$ es la Jacobiana en la solución $x^*$.
*   $E$ es una matriz de 1s de las mismas dimensiones que la Jacobiana
*   Los siguientes parámetros: $maxIter = 2000$, $Tol=10e-14$, $\sigma_1=\sigma_2 = 0.1$, $r=0.1$, $\rho = \sqrt{0.9}$ y $\theta_k=1$, $\epsilon_k = \dfrac{1}{(k+1)^2}$ para cada $k$ en las iteraciones.

1. Esta es una variante de la función de Rosenbrock. $$F_1(\mathbf{x})=\begin{pmatrix}
10(x_2 - x_1^2)\\
1-x_1
\end{pmatrix}
$$ cuyo óptimo es $x^*=(1,1)^t$.

In [ ]:
x_ = np.array([1.0, -5.0])
beta = [1.0,5.0,10.0]
B = [JF1(np.zeros(2)) + np.ones([2,2]), JF1(np.zeros(2))+ 10*np.ones([2,2]), np.identity(2), 10*np.identity(2)]
Bn = ["Jx*+E", "Jx*+10E", "I", "10I"]
print_results(beta, 10e-14, 2000, B, Bn, F1, "F1", x_)

+-------------------------------------------------------------------------------------------------------------------------------+
|                                            Resultados del método de Broyden con F1                                            |
+------+---------+-----------+------------------------+-----------+------------------------+-----------+------------------------+
| beta |   B_0   | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       |
+------+---------+-----------+------------------------+-----------+------------------------+-----------+------------------------+
| 1.0  |  Jx*+E  |     13    | 1.7763568394002505e-14 |     16    | 2.2232198742534223e-15 |     12    |          0.0           |
| 5.0  |  Jx*+E  |     11    | 2.7755575615628914e-14 |     19    | 9.994474079390603e-15  |     17    | 1.1102230246251565e-14 |
| 10.0 |  Jx*+E  |     11    | 2.6645352591003757e-14 |     21    | 3.3472809927562375e-15

2. Función singular de Powell$$F_2(\mathbf{x})=\begin{pmatrix}
x_1 +10x_2\\
5^{\frac{1}{2}}(x_3-x_4) \\
(x_2 - 2x_3)^2 \\
10^{\frac{1}{2}}(x_1-x_4)^2
\end{pmatrix}$$
cuyo óptimo es en $x*=(0,0,0,0)^t$.

In [ ]:
x_ = np.array([3.0, -1.0, 0, 1.0])
beta = [1.0, -1.0, 0.01, -0.01]
B = [np.identity(4), 10*np.identity(4), np.identity(4)+np.ones([4,4])]
Bn = ["I", "10I", "I+E"]
print_results(beta, 10e-14, 2000, B, Bn, F2, "F2", x_)

+----------------------------------------------------------------------------------------------------------------------------+
|                                          Resultados del método de Broyden con F2                                           |
+-------+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+
|  beta | B_0 | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       |
+-------+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+
|  1.0  |  I  |    2000   |  0.01873033205367971   |    2000   |  0.00483344199091699   |    2000   |  0.005636425989594949  |
|  -1.0 |  I  |    387    | 1.2056892443013643e-14 |    2000   | 1.9198628509254815e-05 |    2000   |  2.36593298238335e-06  |
|  0.01 |  I  |    2000   | 1.4897354630228884e-09 |    2000   | 2.1048761390194785e-08 |    2000   | 2.9171238

Para estas siguientes pruebas, se están considerando los mismos parámetros que en los casos anteriores, salvo lo siguiente:
*   $maxIter=10000$
*   $Tol = 10e-10$
*   La inicialización para $x_0$ sigue siendo de la forma $\beta \hat{x}$, pero ahora, $\hat{x}$ se inicia como sigue: $$\hat{x}=\left(1, \frac{1}{2},...,\frac{1}{n}\right)^t$$donde $n$ es la dimensión a probar.

3. La versión discretizada de la ecuación-H de Chandrasekhar: $$F_{3_i}(\mathbf{x}) = x_i -\left( 1-\dfrac{c}{2n}\displaystyle\sum_{j=1}^n \dfrac{\mu_ix_j}{\mu_i + \mu_j} \right)^{-1}$$ para $i=1,...,n$ y $\mu_i = \dfrac{i-0.5}{n}$. $c\in [0,1]$, para las pruebas se usa $c=1$.

In [ ]:
n = [10, 50, 100]
beta = [1, 10, -10, -1]
print_results1(beta, 10e-10, int(10e+3), F3, "F3", n)

+--------------------------------------------------------------------------------------------------------------------------------------+
|                                               Resultados del método de Broyden con F3                                                |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| beta | B_0 |  n  | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       | Nd |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
|  1   |  I  |  10 |     24    | 4.4308880592903185e-10 |     24    | 4.430906736596689e-10  |     17    | 7.796975402733466e-10  | 16 |
|  1   |  I  |  50 |     25    | 4.2041183812197206e-10 |     25    | 4.2041336350494786e-10 |     19    | 2.949366323078279e-10  | 16 |
|  1   |  I  | 100 |     25    | 6.068454

4. Problema de valores de frontera discretizado: $$F_4(\mathbf{x})=\begin{pmatrix}
2 & -1 & 0 & \cdots & 0\\
-1 & 2 & -1 & \cdots & 0\\
0 & -1 & 2 & \cdots & 0\\
\vdots & \vdots & \vdots & \cdots & \vdots\\
0 & 0 & 0 & \cdots & 2
\end{pmatrix}\mathbf{x} + \dfrac{1}{(n+1)^2}(\sin(x_1)-1,...,\sin(x_n)-1)^t$$

In [ ]:
n = [10, 50, 100]
beta = [0.1, 1.0, 10, -10]
print_results1(beta, 10e-10, int(10e+3), F4, "F4", n)

+--------------------------------------------------------------------------------------------------------------------------------------+
|                                               Resultados del método de Broyden con F4                                                |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| beta | B_0 |  n  | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       | Nd |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| 0.1  |  I  |  10 |     20    | 5.961618672577028e-10  |     26    | 6.668711315840854e-10  |     22    | 9.800098929661543e-12  | 10 |
| 0.1  |  I  |  50 |    108    | 8.539659535701317e-10  |    197    | 4.558787738805605e-10  |    215    | 8.863632276484985e-10  | 15 |
| 0.1  |  I  | 100 |    262    | 2.941207

5. La función dada por:
$$F_{5_i}(\mathbf{x}) = \begin{cases}
2x_1 + \sin(x_1)-1 & \text{ si }i=1, \\
-2x_{i-1} + 2x_i + \sin(x_i) -1 & \text{ si }i=2,...,n-1 \\
2x_n +\sin(x_n) -1 & \text{ si }i=n
\end{cases}$$

In [ ]:
n = [10, 50, 100, 500]
beta = [1.0, 10, -10, -0.1]
print_results1(beta, 10e-10, int(10e+3), F5, "F5", n)

+--------------------------------------------------------------------------------------------------------------------------------------+
|                                               Resultados del método de Broyden con F5                                                |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| beta | B_0 |  n  | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       | Nd |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| 1.0  |  I  |  10 |     26    | 2.0485210626375759e-10 |     25    | 1.6747457860684494e-10 |     20    | 5.693920006035944e-11  | 9  |
| 1.0  |  I  |  50 |    119    | 4.4922390903368577e-10 |     78    | 6.430680723000029e-10  |     66    | 1.2843312269021646e-10 | 16 |
| 1.0  |  I  | 100 |    1479   | 4.958859

6. La función dada por:
$$F_{6_i}(\mathbf{x}) = \begin{cases}
e^{x_1} -1 & \text{ si }i=1, \\
\dfrac{i}{10}(e^{x_i} + x_{i-1} -1) & \text{ si }i=2,...,n
\end{cases}$$

In [ ]:
n = [10, 50, 100, 200, 500]
beta = [-0.01, 0.01, -0.1, 0.1]
print_results1(beta, 10e-10, int(10e+3), F6, "F6", n)

<ipython-input-49-e49ba56da7ca>:51: RuntimeWarning: overflow encountered in exp
  f[i-1] = i*(np.exp(x[i-1]) + x[i-2] -1)/10
/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
<ipython-input-49-e49ba56da7ca>:51: RuntimeWarning: overflow encountered in scalar multiply
  f[i-1] = i*(np.exp(x[i-1]) + x[i-2] -1)/10
/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


+---------------------------------------------------------------------------------------------------------------------------------------+
|                                                Resultados del método de Broyden con F6                                                |
+-------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
|  beta | B_0 |  n  | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       | Nd |
+-------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+------------------------+----+
| -0.01 |  I  |  10 |     21    | 3.706667308638638e-10  |     26    | 1.6953350949095357e-10 |     16    |  6.33167150082655e-10  | 10 |
| -0.01 |  I  |  50 |     90    | 7.879577260494137e-10  |     87    | 5.339034145285423e-10  |     78    | 9.036178868651236e-10  | 12 |
| -0.01 |  I  | 100 |   10000   | 

Para esta siguiente prueba, solo cambiamos la forma de $\hat{x}$ para que sea únicamente un vector de 1s.
6. La función dada por:
$$F_{6_i}(\mathbf{x}) = \begin{cases}
x_1(x_1^2 +x_2^2) -1 & \text{ si }i=1, \\
x_i(x_{i-1}^2 +2x_i^2 +x_{i+1}^2) -1 & \text{ si }i=2,...,n-1 \\
x_n(x_{n-1}^2 +x_n^2) & \text{ si }i=n
\end{cases}$$

In [ ]:
n = [10, 50, 100]
beta = [-15, -10, -5, 5, 10]
print_results1(beta, 10e-10, int(10e+3), F7, "F7", n, 2)

Terminamos por singularidad de matriz


<ipython-input-49-e49ba56da7ca>:58: RuntimeWarning: overflow encountered in scalar multiply
  f[-1] = x[-1]*(x[-2]**2 + x[-1]**2)
<ipython-input-49-e49ba56da7ca>:60: RuntimeWarning: overflow encountered in scalar multiply
  f[i] = x[i]*(x[i-1]**2 +2*(x[i]**2) +x[i+1]**2)-1


Terminamos por singularidad de matriz


<ipython-input-49-e49ba56da7ca>:57: RuntimeWarning: overflow encountered in scalar multiply
  f[0] = x[0]*(x[0]**2 + x[1]**2) - 1


Terminamos por singularidad de matriz
Terminamos por singularidad de matriz
Terminamos por singularidad de matriz
+----------------------------------------------------------------------------------------------------------------------------------------+
|                                                Resultados del método de Broyden con F7                                                 |
+------+-----+-----+-----------+-------------------------+-----------+------------------------+-----------+------------------------+-----+
| beta | B_0 |  n  | M1 - Iter |       M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||       |  Nd |
+------+-----+-----+-----------+-------------------------+-----------+------------------------+-----------+------------------------+-----+
| -15  |  I  |  10 |     21    |  2.2273490830966243e+71 |    643    |  9.95264057424165e-10  |    483    | 1.5106520787807572e-10 |  28 |
| -15  |  I  |  50 |   10000   |           nan      

In [ ]:
n = [12, 21, 37, 72, 81, 144, 256]
beta = [15]
print_results1(beta, 10e-10, int(10e+3), F7, "F7", n, 2)

Terminamos por singularidad de matriz
Terminamos por singularidad de matriz


<ipython-input-49-e49ba56da7ca>:57: RuntimeWarning: overflow encountered in scalar multiply
  f[0] = x[0]*(x[0]**2 + x[1]**2) - 1
<ipython-input-49-e49ba56da7ca>:58: RuntimeWarning: overflow encountered in scalar multiply
  f[-1] = x[-1]*(x[-2]**2 + x[-1]**2)
<ipython-input-49-e49ba56da7ca>:60: RuntimeWarning: overflow encountered in scalar multiply
  f[i] = x[i]*(x[i-1]**2 +2*(x[i]**2) +x[i+1]**2)-1


+-------------------------------------------------------------------------------------------------------------------------------------+
|                                               Resultados del método de Broyden con F7                                               |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+-----------------------+----+
| beta | B_0 |  n  | M1 - Iter |      M1 - ||fk||       | M2 - Iter |      M2 - ||fk||       | M3 - Iter |      M3 - ||fk||      | Nd |
+------+-----+-----+-----------+------------------------+-----------+------------------------+-----------+-----------------------+----+
|  15  |  I  |  12 |     24    | 2.4384277658984924e+77 |    420    | 2.535661436897076e-11  |    304    | 8.779404798992073e-10 | 36 |
|  15  |  I  |  21 |     12    | 4.248844123103982e+45  |    355    | 9.758712280254915e-10  |    254    | 6.836308867741627e-10 | 35 |
|  15  |  I  |  37 |   10000   |          nan   

# Conclusiones
Los resultados mostraron mejoras significativas en la estabilidad y rendimiento del método modificado, especialmente en funciones propensas a desbordamientos numéricos. Este avance ofrece una alternativa para problemas complejos donde los métodos tradicionales podrían fallar. La búsqueda en línea libre de derivadas contribuyó a la convergencia global y superlineal, validando la efectividad de las modificaciones propuestas.

## Referencias:
1. Li, D. H., & Fukushima, M. (2000). A derivative-free line search and global convergence of Broyden-like method for nonlinear equations. Optimization Methods and Software, 13(3), 181–201. https://doi.org/10.1080/10556780008805782
2. Weijun Zhou, Li Zhang, A modified Broyden-like quasi-Newton method for nonlinear equations, Journal of Computational and Applied Mathematics, Volume 372, 2020, 112744, ISSN 0377-0427, https://doi.org/10.1016/j.cam.2020.112744.

<img src="https://imgb.ifunny.co/images/7c921a5d3d0f7d585591e0b157578f1ddcf99b2fa4cf2b88c8727ebf634fb76a_1.webp" alt="Image Title">
